# Options Surface NaN Analysis

This notebook scans the derived options surface outputs, loads each parquet with pandas, and summarizes missing-data patterns. Use it to verify which columns contain NaN values across multiple symbols and time ranges.

In [4]:
from pathlib import Path
import os
from collections import Counter, defaultdict
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
pd.set_option('display.max_rows', 100)

In [5]:
def _resolve_repo_root(start: Path) -> Path:
    current = start
    for _ in range(10):
        if (current / '.git').exists():
            return current
        if current.parent == current:
            break
        current = current.parent
    return start
REPO_ROOT = _resolve_repo_root(Path.cwd())
DATA_ROOT = Path(os.environ.get('QUANTO_DATA_ROOT', REPO_ROOT / '.quanto_data'))
SURFACE_ROOT = DATA_ROOT / 'derived' / 'options_surface_v1'
SURFACE_ROOT

PosixPath('/Users/ahmed/PycharmProjects/Quanto/.quanto_data/derived/options_surface_v1')

In [6]:
if not SURFACE_ROOT.exists():
    raise FileNotFoundError(f'No derived options surface data found under {SURFACE_ROOT}')
files = sorted(SURFACE_ROOT.glob('*/*/*.parquet'))
len(files)

80

In [7]:
nan_counts = Counter()
symbol_nan_counts = defaultdict(set)
total_rows = 0
for path in files:
    df = pd.read_parquet(path)
    if df.empty:
        continue
    total_rows += len(df)
    counts = df.isna().sum()
    symbol = path.parent.parent.name
    for col, cnt in counts.items():
        if cnt:
            nan_counts[col] += int(cnt)
            symbol_nan_counts[col].add(symbol)
sum(nan_counts.values())

165718

In [8]:
summary_rows = []
for col, cnt in nan_counts.items():
    summary_rows.append({
        'column': col,
        'nan_count': cnt,
        'nan_pct': cnt / total_rows if total_rows else 0.0,
        'symbols_with_nan': len(symbol_nan_counts[col]),
    })
nan_summary = pd.DataFrame(summary_rows).sort_values('nan_count', ascending=False)
nan_summary.head(20)

,column,nan_count,nan_pct,symbols_with_nan
5,OPT:HV:60,28740,0.988988,20
13,OPT:IVX:TERM_SLOPE_30_180,12869,0.442842,20
14,OPT:IVX:TERM_SLOPE_30_90,12163,0.418548,20
8,OPT:IVP:150,9448,0.325120,20
9,OPT:IVP:180,9447,0.325086,20
7,OPT:IVP:120,9446,0.325052,20
3,OPT:HV:180,9398,0.323400,20
12,OPT:IVP:90,9371,0.322471,20
10,OPT:IVP:30,9337,0.321301,20
11,OPT:IVP:60,9330,0.321060,20


In [ ]:
top_k = nan_summary.head(20) if not nan_summary.empty else pd.DataFrame()
if not top_k.empty:
    fig, ax = plt.subplots(figsize=(10,6))
    ax.barh(top_k['column'], top_k['nan_count'])
    ax.invert_yaxis()
    ax.set_xlabel('NaN Count')
    ax.set_title('Top Columns by NaN Count')
    plt.show()
else:
    print('No NaN values detected in the loaded data.')

In [ ]:
if not nan_summary.empty:
    display(nan_summary.sort_values('symbols_with_nan', ascending=False).head(20))